In [1]:
import os

In [2]:
os.chdir("../")
%pwd


'c:\\Projects\\Chicken_disease_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml_file, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filePath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filePath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        eval_config = ModelEvaluationConfig(
            model_path = self.config.training.trained_model_filepath,
            training_data = "artifacts\data_ingestion\Chicken-fecal-images",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
        )
        
        return eval_config

In [6]:
from urllib.parse import urlparse
import tensorflow as tf

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle= False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.models:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self.valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(out_filepath=Path("score.json"), data=scores)

In [8]:
try:
    config_manager = ConfigurationManager()
    eval_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(eval_config)
    model_evaluation.evaluation()
    model_evaluation.save_score()
except Exception as e:
    raise e

[2024-12-01 12:37:17,480: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-01 12:37:17,487: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-01 12:37:17,491: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.
5/5 [==============================] - 10s 2s/step - loss: 16.2067 - accuracy: 0.5000
[2024-12-01 12:37:28,805: INFO: common: json file saved at: score.json]
